In [26]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch import utils
import scipy.ndimage as ndimage
from PIL import Image
from torchvision import transforms

In [30]:
class Parameters:
    def __init__(self):
        self.image_dir = "../WFLW_images/"
        self.train_annotations = "../WFLW_annotations/list_98pt_rect_attr_train_test/list_98pt_rect_attr_train.txt"
        self.test_annotations = "../WFLW_annotations/list_98pt_rect_attr_train_test/list_98pt_rect_attr_test.txt"
        self.data_transform = transforms.Compose([Resize(224), Normalize(), ToTensor()])

In [56]:
class keypoint_dataset(Dataset):
    
    def __init__(self, image_dir, annotation_file, transforms = None):
        self.keypoints = pd.read_csv(annotation_file)
        self.image_dir = image_dir
        self.transforms = transforms
        
    def __len__(self):
        return len(self.keypoints)
    
    def __getitem__(self, idx):
        print(self.keypoints[idx, :])
        image_name = self.keypoints.iloc[idx, 206]
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert("L")
        keypoints = self.keypoints.iloc[idx, 0:196].as_matrix()
        keypoints = keypoints.astype('float').reshape(-1, 2)
        dataset = {'image':image, 'keypoints':keypoint}
        if self.transforms:
            image = self.transforms(image)
        return dataset

In [57]:
class Normalize(object):
    
    def __call__(self, sample):
        image, key_pts = sample['image'], sample['keypoints']
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        image = image/255.0
        
        key_pts = (key_pts - 100)/50.0 # assume: mean = 100, std = 50
        return {'image': image, 'keypoints': key_pts}

In [58]:
class Resize(object):
    # Arg: Desired output size 
    #   --> if tupple : returns output of the tuple size
    #   --> if int : Smaller Edge = given int
    #                Longer Edge = given int * (orig smaller edge / orig longer edge)
    #                (keeps the original ratio the same)
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size
    
    def __call__(self, sample):
        image, key_pts = sample['image'], sample['keypoints']
        h, w = image.shape[:2]
        
        if(isinstance(self.output_size, tuple)):
            new_h, new_w = self.output_size
        else:
            if(h <= w):
                new_h, new_w = self.output_size, self.output_size*(w/h)
            else:
                new_h, new_w = self.output_size*(h/w), self.output_size
        
        new_h, new_w = int(new_h), int(new_w)
        new_image = cv2.resize(image, (new_w, new_h))
        #  "*" : [np.arr1 * np.arr2] :
        #           (1) Both arrays have SAME dimension: ELEMENTWISE MULTIPLICAION 
        #           (2) Same # of columns & 1 array has 1 row :  
        #               ELEMENTWISE MULTIPLICAION w/ that 1 row with every row of the longer arr
        
        new_key_pts = key_pts * [new_w/w, new_h/h]
        
        return { 'image' : new_image, 'keypoints' : new_key_pts }   

In [59]:
class ToTensor(object):
        
    def __call__(self, sample):
        image, key_pts = sample['image'], sample['keypoints']
        
        if(len(image.shape) == 2): # if image has NO 3rd grayscale channel (1)
            image = image.reshape(image.shape[0], image.shape[1], 1)
            
        # Put grayscale channel at the 1st dimension, because:
        # Numpy image: [H, W, C]  &  Tensor image: [C, H, W]
        image = image.transpose((2, 0, 1)) # make (0, 1, 2) --> (2, 0, 1)
        return {'image': torch.from_numpy(image), 'keypoints': torch.from_numpy(key_pts)}

In [60]:
parameters = Parameters()
train_dataset = keypoint_dataset(parameters.image_dir, parameters.train_annotations, parameters.data_transform)
test_dataset = keypoint_dataset(parameters.image_dir, parameters.test_annotations, parameters.data_transform)

In [61]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=128,num_workers=0,shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=128,num_workers=0,shuffle=True)

In [62]:
for d in enumerate(train_dataloader):
    plt.imshow(d['image'])

TypeError: unhashable type: 'slice'